In [14]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest # 特征选择
from sklearn.feature_selection import chi2 # 卡方统计量

from sklearn.preprocessing import MinMaxScaler # 数据归一化
from sklearn.decomposition import PCA # 主成分分析
from sklearn.model_selection import GridSearchCV # 网格搜索交叉验证,用于选择最优的参数

In [2]:
## 设置属性防止中文乱码
mpl.rcParams['font.sans-serif'] = [u'SimHei']
mpl.rcParams['axes.unicode_minus'] = False

In [5]:
warnings.filterwarnings('ignore',category=FutureWarning)

In [7]:
iris_feature_E = 'sepal length', 'sepal width', 'petal length', 'petal width'
iris_feature_C = '花萼长度', '花萼宽度', '花瓣长度', '花瓣宽度'
iris_class = 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'

In [13]:
# 读取数据,查看数据
data=pd.read_csv("datas/iris.data",header=None)
# 获取X变量
x=data[[0,1,2,3]]
y=pd.Categorical(data[4]).codes
print("总样本数目: %d ; 特征属性数目: %d" % x.shape)
print(x.head())
print(type(y))
print(np.unique(y))

总样本数目: 150 ; 特征属性数目: 4
     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
<class 'numpy.ndarray'>
[0 1 2]


In [15]:
# 训练集,测试集划分
x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=14)

In [30]:
x_train,x_test,y_train,y_test=x_train1,x_test1,y_train1,y_test1
print("训练数据集样本数目: %d,测试数据集样本数目: %d" % (x_train.shape[0],x_test.shape[0]))

# 决策树的叶子节点的目标值必须是数值型变量
y_train=y_train.astype(np.int)
y_test=y_test.astype(np.int)

print(y_train.dtype)
print(y_train[:10])

训练数据集样本数目: 120,测试数据集样本数目: 30
int64
[0 1 1 0 1 0 2 1 2 1]


# 数据标准化

## StandardScaler (基于特征矩阵的列,将属性值转换至服从正态分布)

1. 标准化是依照特征矩阵的列处理数据,其通过求z-score的方法,将样本的特征值转换到同一量纲下
2. 常用于基于正态分布的算法,比如回归算法

# 数据归一化

## MinMaxScaler(区间缩放,基于最大最小值,将数据转换到0,1区间上的)

1. 提升模型收敛速度,提升模型精度
2. 常见于神经网络

# Normalizer(基于矩阵的行,将样本向量转换为单位向量)

1. 其目的在于样本向量在点乘运算或其他核函数计算相似性时,拥有统一的标准
2. 常见用于文本分类和聚类,logistic回归中也会使用,有效防止过拟合


In [35]:
ss=MinMaxScaler()
# sklearn 中模型API说明:
# fit : 模型训练,基于给定的训练集(X,Y)训练出一个模型;该API是没有返回值的.eg:ss.fit(X_train,Y_train)执行后,ss这个模型对象就训练好了.
# transfrom : 数据转换,使用训练好的模型对给定的数据集(X) 进行转换操作;一般如果训练集进行转换操作,那么测试集也需要转换操作;这个API只在特征工程中出现
# predict : 数据转换/数据预测,功能和transfrom一样,都是对给定的数据集x进行转换操作,只是transfrom中返回的是一个新的x,而predict返回的是预测值;这个API只在算法模型中出现
# fit_transfrom : fit+transfrom两个API的合并,表示先根据给定的数据训练模型(fit),然后使用训练好的模型对给定的数据进行转换操作
x_train=ss.fit_transform(x_train)

x_test=ss.transform(x_test)

print("原始数据各个特征属性的调整最小值:",ss.min_)
print("原始数据各个特征属性的缩放数据值:",ss.scale_)

原始数据各个特征属性的调整最小值: [0. 0. 0.]
原始数据各个特征属性的缩放数据值: [1. 1. 1.]


In [33]:
# 特征选择: 从已有的特征中选择出影响目标值最大的特征属性
# 常用方法:
# 分类:F统计量,卡方系数,互信息mutual_info_classif
# 连续:皮尔逊相关系数,F统计量,互信息mutual_info_classif

# SelectKBest(卡方系数)
# 在当前的案例中,使用SelectKBest这个方法从4个原始特征属性,选择出来3个
ch2=SelectKBest(chi2,k=3)
# k默认为10,如果指定了k,那么就会返回指定的特征个数
x_train=ch2.fit_transform(x_train,y_train) # 训练并转换
x_test=ch2.transform(x_test) # 转换

select_name_index=ch2.get_support(indices=True)
print("对类别判断影响最大的三个特征属性分布是:",ch2.get_support(indices=False))
print(select_name_index)

对类别判断影响最大的三个特征属性分布是: [ True False  True  True]
[0 2 3]


In [34]:
# 降维:对于数据而言,如果特征属性比较多,在构建过程中,会比较复杂,这个时候考虑将多维(高维)映射到低维的数据
# 常用的方法:
# PCA:主成分分析(无监督)
# LDA:线性判别分析(有监督)类内方差最小,人脸识别,通常先做一次pca
pca=PCA(n_components=2) # 构建一个pca对象,设置最终维度是2维

array([[0.22222222, 0.06896552, 0.04166667],
       [0.19444444, 0.37931034, 0.375     ],
       [0.5       , 0.62068966, 0.45833333],
       [0.41666667, 0.01724138, 0.04166667],
       [0.36111111, 0.48275862, 0.41666667],
       [0.33333333, 0.03448276, 0.04166667],
       [0.44444444, 0.68965517, 0.70833333],
       [0.33333333, 0.46551724, 0.41666667],
       [0.47222222, 0.67241379, 0.58333333],
       [0.58333333, 0.5862069 , 0.58333333],
       [0.80555556, 0.84482759, 0.70833333],
       [0.22222222, 0.0862069 , 0.04166667],
       [0.38888889, 0.06896552, 0.125     ],
       [0.41666667, 0.48275862, 0.45833333],
       [0.58333333, 0.72413793, 0.75      ],
       [0.5       , 0.77586207, 0.54166667],
       [0.16666667, 0.06896552, 0.        ],
       [0.02777778, 0.05172414, 0.04166667],
       [0.02777778, 0.03448276, 0.04166667],
       [0.55555556, 0.56896552, 0.5       ],
       [0.19444444, 0.05172414, 0.04166667],
       [0.19444444, 0.0862069 , 0.125     ],
       [0.